# Table of content:
01. Importing the Data and Libraries
02. Security Implications: Addressing Personally Identifiable Information (PII)
03. Creating a regional segmentation of the data - “Region” column based on the “State” column from the customer's data set.
04. Creating an exclusion flag for low-activity customers (customers with less than 5 orders) and excluding them from the data.
05. Creating a profiling variable based on age, income, certain goods in the “department_id” column, and number of dependents.

# 1. Importing the Data and Libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Import Data
path = r'/Users/azadeh/Documents/Instacart Basket Analysis'

In [3]:
# Import orders_products_merged_flags
df_ords_prods_merge =  pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data','orders_products_customers_merge.pkl'))

In [4]:
df_ords_prods_merge.head()

,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days since the last order,product_id,add_to_cart_order,reordered,...,order_frequency_flag,first_name,surname,gender,state,age,date_joined,dependent_count,fam_status,income
0,0,2539329,1,1,2,8,NaN,196,1,0,...,Non-frequent,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,1,2398795,1,2,3,7,15.0,196,1,1,...,Non-frequent,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,2,473747,1,3,3,12,21.0,196,1,1,...,Non-frequent,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,3,2254736,1,4,4,7,29.0,196,1,1,...,Non-frequent,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,4,431534,1,5,4,15,28.0,196,1,1,...,Non-frequent,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


# 2. Address any PII data

Addressing PII is critically important for ensuring the security and privacy of individuals' data. Here we have First and last name in our dataset which we need to drop it.

In [5]:
#Dropping first_name and surname column from orders_products_customers_merge.pkl
df = df_ords_prods_merge.drop(columns = ['first_name','surname'],axis=1)


In [6]:
df.head()

,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days since the last order,product_id,add_to_cart_order,reordered,...,spending_flag,median_days_order,order_frequency_flag,gender,state,age,date_joined,dependent_count,fam_status,income
0,0,2539329,1,1,2,8,NaN,196,1,0,...,Low spender,20.5,Non-frequent,Female,Alabama,31,2/17/2019,3,married,40423
1,1,2398795,1,2,3,7,15.0,196,1,1,...,Low spender,20.5,Non-frequent,Female,Alabama,31,2/17/2019,3,married,40423
2,2,473747,1,3,3,12,21.0,196,1,1,...,Low spender,20.5,Non-frequent,Female,Alabama,31,2/17/2019,3,married,40423
3,3,2254736,1,4,4,7,29.0,196,1,1,...,Low spender,20.5,Non-frequent,Female,Alabama,31,2/17/2019,3,married,40423
4,4,431534,1,5,4,15,28.0,196,1,1,...,Low spender,20.5,Non-frequent,Female,Alabama,31,2/17/2019,3,married,40423


# 03. Creating a regional segmentation of the data 

In [7]:
# create “Region” column based on the “State” column from the customer's data set.
Northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
Midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']
South = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
West = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [8]:
df.loc[df['state'].isin(Northeast), 'region'] = 'Northeast'
df.loc[df['state'].isin(Midwest), 'region'] = 'Midwest'
df.loc[df['state'].isin(South), 'region'] = 'South'
df.loc[df['state'].isin(West), 'region'] = 'West'

In [9]:
# Checking the "region" column
df['region'].value_counts(dropna = False)

region
South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: count, dtype: int64

Difference in spending habits between the different U.S. regions. (Hint: You can do this by crossing the variable you just created with the spending flag.)

In [10]:
#crossing no.of clients in different regions with "spending flag"
crosstab = pd.crosstab(df['spending_flag'], df['region'], dropna = False)

In [11]:
crosstab.to_clipboard()

In [12]:
crosstab

region,Midwest,Northeast,South,West
spending_flag,,,,
High spender,2497034,1875170,3549625,2722093
Low spender,5100291,3847566,7242260,5570820


In each region, the number of low spenders is significantly higher than the number of high spenders. This indicates that a larger portion of the population falls into the category of low spenders.

# 04. Creating an exclusion flag for low-activity customers (customers with less than 5 orders) and excluding them from the data.

In [13]:
# Create the Exclusion Flag column with "low activity" and "active customer"
df.loc[df['max_order'] < 5, 'exclusion_flag'] = 'low activity'
df.loc[df['max_order'] >= 5, 'exclusion_flag'] = 'active customer'

In [14]:
#Check frequency
df['exclusion_flag'].value_counts(dropna=False)

exclusion_flag
active customer    30964564
low activity        1440295
Name: count, dtype: int64

In [15]:
#create subset for low activity
df_low_activity= df[df['exclusion_flag'] =='low activity']


In [16]:
df_active_customer = df[df['exclusion_flag'] =='active customer']

In [17]:
df_active_customer.head()

,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days since the last order,product_id,add_to_cart_order,reordered,...,order_frequency_flag,gender,state,age,date_joined,dependent_count,fam_status,income,region,exclusion_flag
0,0,2539329,1,1,2,8,NaN,196,1,0,...,Non-frequent,Female,Alabama,31,2/17/2019,3,married,40423,South,active customer
1,1,2398795,1,2,3,7,15.0,196,1,1,...,Non-frequent,Female,Alabama,31,2/17/2019,3,married,40423,South,active customer
2,2,473747,1,3,3,12,21.0,196,1,1,...,Non-frequent,Female,Alabama,31,2/17/2019,3,married,40423,South,active customer
3,3,2254736,1,4,4,7,29.0,196,1,1,...,Non-frequent,Female,Alabama,31,2/17/2019,3,married,40423,South,active customer
4,4,431534,1,5,4,15,28.0,196,1,1,...,Non-frequent,Female,Alabama,31,2/17/2019,3,married,40423,South,active customer


In [18]:
#exporting active_customer dataframe
df_active_customer.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'active_customer.pkl'))

The marketing and business strategy units at Instacart want to create more-relevant marketing strategies for different products and are, thus, curious about customer profiling in their database. Create a profiling variable based on age, income, certain goods in the “department_id” column, and number of dependents. You might also use the “orders_day_of_the_week” and “order_hour_of_day” columns if you can think of a way they would impact customer profiles. (Hint: As an example, try thinking of what characteristics would lead you to the profile “Single adult” or “Young parent.”)

# 05. Creating a profiling variable based on (age, income, certain goods in the “department_id” column and number of dependents)

In [19]:
# Create age profile
# Check descriptive statistics for age
df_active_customer['age'].describe()

count    3.096456e+07
mean     4.946803e+01
std      1.848528e+01
min      1.800000e+01
25%      3.300000e+01
50%      4.900000e+01
75%      6.500000e+01
max      8.100000e+01
Name: age, dtype: float64

In [20]:
# create flag for young adult (18-33)
df_active_customer.loc[(df_active_customer['age'] >= 18) & (df_active_customer['age'] <= 33), 'age_group'] = 'Young adult (18-33)'


/var/folders/4h/rfmmvwhd05z_ywjwzkv5fv7c0000gn/T/ipykernel_68831/756678533.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_active_customer.loc[(df_active_customer['age'] >= 18) & (df_active_customer['age'] <= 33), 'age_group'] = 'Young adult (18-33)'


In [21]:
# create flag for adult (34-49)
df_active_customer.loc[(df_active_customer['age'] >= 34) & (df_active_customer['age'] <= 49), 'age_group'] = 'Adult (34-49)'

In [22]:
# create flag for Middle_Aged adult (50-64)
df_active_customer.loc[(df_active_customer['age'] >= 50) & (df_active_customer['age'] <= 64), 'age_group'] = 'Middle-Aged Adult(50-64)'

In [23]:
# create flag for Senior Adult (65+)
df_active_customer.loc[(df_active_customer['age'] >= 65), 'age_group'] = 'Senior Adult (65+)'

In [24]:
# check age_group frequency
df_active_customer['age_group'].value_counts(dropna=False)

age_group
Senior Adult (65+)          8195544
Adult (34-49)               7790824
Young adult (18-33)         7757465
Middle-Aged Adult(50-64)    7220731
Name: count, dtype: int64

In [25]:
# Create income profile
# Check descriptive statistics for income
df_active_customer['income'].describe()

count    3.096456e+07
mean     9.967587e+04
std      4.314187e+04
min      2.590300e+04
25%      6.729200e+04
50%      9.676500e+04
75%      1.281020e+05
max      5.939010e+05
Name: income, dtype: float64

In [26]:
# create flag for Low income
df_active_customer.loc[(df_active_customer['income'] >= 25000) & (df_active_customer['income'] <= 67000), 'income_group'] = 'Low income'

/var/folders/4h/rfmmvwhd05z_ywjwzkv5fv7c0000gn/T/ipykernel_68831/1999221987.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_active_customer.loc[(df_active_customer['income'] >= 25000) & (df_active_customer['income'] <= 67000), 'income_group'] = 'Low income'


In [27]:
# create flag for Medium income
df_active_customer.loc[(df_active_customer['income'] >= 67001) & (df_active_customer['income'] <= 128000), 'income_group'] = 'Medium income'

In [28]:
# create flag for High income
df_active_customer.loc[(df_active_customer['income'] >= 128001), 'income_group'] = 'High income'

In [29]:
# check income_group frequency
df_active_customer['income_group'].value_counts(dropna=False)

income_group
Medium income    15549969
High income       7753117
Low income        7661478
Name: count, dtype: int64

In [30]:
# Create fam_status profile
# Check descriptive statistics for fam_status
df_active_customer['fam_status'].value_counts(dropna=False)

fam_status
married                             21743711
single                               5094410
divorced/widowed                     2645271
living with parents and siblings     1481172
Name: count, dtype: int64

In [31]:
df_active_customer.columns

Index(['Unnamed: 0_x', 'order_id', 'user_id', 'order_number',
       'orders_day_of_week', 'order_hour_of_day', 'days since the last order',
       'product_id', 'add_to_cart_order', 'reordered', 'Unnamed: 0_y',
       'product_name', 'aisle_id', 'department_id', 'prices', '_merge',
       'price_range_loc', 'busiest_day', 'busiest_days',
       'busiest_period_of_day', 'max_order', 'loyalty_flag', 'average_price',
       'spending_flag', 'median_days_order', 'order_frequency_flag', 'gender',
       'state', 'age', 'date_joined', 'dependent_count', 'fam_status',
       'income', 'region', 'exclusion_flag', 'age_group', 'income_group'],
      dtype='object')

In [32]:
# check frequency of dependent_count
df_active_customer['dependent_count'].value_counts(dropna=False)

dependent_count
3    7772516
0    7739681
2    7733261
1    7719106
Name: count, dtype: int64

In [33]:
# create crosstab between fam_status and dependent_count
fam_profile = pd.crosstab(df_active_customer['fam_status'], df_active_customer['dependent_count'], dropna = False)

In [34]:
fam_profile

dependent_count,0,1,2,3
fam_status,,,,
divorced/widowed,2645271,0,0,0
living with parents and siblings,0,508045,484658,488469
married,0,7211061,7248603,7284047
single,5094410,0,0,0


By looking at this crosstab, we can create Family_characteristic profile by naming :

In [35]:
#Create family_charac profile 
#(Single Adult: 0 dependents or single','divorced/widowed , Parent(married): 1-3 dependents, Young Adult(Living with parents and sibling): 1-3 dependents

In [36]:
# Check descriptive statistics for dependent_count
df_active_customer['dependent_count'].value_counts(dropna=False)

dependent_count
3    7772516
0    7739681
2    7733261
1    7719106
Name: count, dtype: int64

In [37]:
# create flag for Single Adult: 0 dependents
df_active_customer.loc[(df_active_customer['fam_status'].isin(['single','divorced/widowed']), 'family_charac')] = 'Single Adult'

/var/folders/4h/rfmmvwhd05z_ywjwzkv5fv7c0000gn/T/ipykernel_68831/678047736.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_active_customer.loc[(df_active_customer['fam_status'].isin(['single','divorced/widowed']), 'family_charac')] = 'Single Adult'


In [38]:
# create flag for Young Parent: Married
df_active_customer.loc[(df_active_customer['fam_status'].isin(['married']), 'family_charac')] = 'Parent'

In [39]:
# create flag for Young Adult: living with parents and siblings
df_active_customer.loc[(df_active_customer['fam_status'].isin(['living with parents and siblings']), 'family_charac')] = 'Young Adult'

In [40]:
# check frequency of family_charac
df_active_customer['family_charac'].value_counts(dropna=False)

family_charac
Parent          21743711
Single Adult     7739681
Young Adult      1481172
Name: count, dtype: int64

In [41]:
# create crosstab between family_charac and age_group
age_char = pd.crosstab(df_active_customer['family_charac'], df_active_customer['age_group'], dropna = False)

In [42]:
age_char

age_group,Adult (34-49),Middle-Aged Adult(50-64),Senior Adult (65+),Young adult (18-33)
family_charac,,,,
Parent,5856214,5377317,6173586,4336594
Single Adult,1934610,1843414,2021958,1939699
Young Adult,0,0,0,1481172


In [43]:
#create certain goods in the “department_id” column
# Import departments_wrangled
df_dept =  pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data','departments_wrangled.csv'))

In [44]:
df_dept.head(21)

,Unnamed: 0,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [45]:
# Rename colunm for 'department_id'

df_dept = df_dept.rename(columns = {'Unnamed: 0' : 'department_id'})

In [46]:
df_active_customer.columns

Index(['Unnamed: 0_x', 'order_id', 'user_id', 'order_number',
       'orders_day_of_week', 'order_hour_of_day', 'days since the last order',
       'product_id', 'add_to_cart_order', 'reordered', 'Unnamed: 0_y',
       'product_name', 'aisle_id', 'department_id', 'prices', '_merge',
       'price_range_loc', 'busiest_day', 'busiest_days',
       'busiest_period_of_day', 'max_order', 'loyalty_flag', 'average_price',
       'spending_flag', 'median_days_order', 'order_frequency_flag', 'gender',
       'state', 'age', 'date_joined', 'dependent_count', 'fam_status',
       'income', 'region', 'exclusion_flag', 'age_group', 'income_group',
       'family_charac'],
      dtype='object')

In [47]:
#merge df_depts with df_active_customer on department_id

df_profiles = df_active_customer.merge(df_dept, on = 'department_id')

In [48]:
df_profiles.head()

,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days since the last order,product_id,add_to_cart_order,reordered,...,date_joined,dependent_count,fam_status,income,region,exclusion_flag,age_group,income_group,family_charac,department
0,0,2539329,1,1,2,8,NaN,196,1,0,...,2/17/2019,3,married,40423,South,active customer,Young adult (18-33),Low income,Parent,beverages
1,1,2398795,1,2,3,7,15.0,196,1,1,...,2/17/2019,3,married,40423,South,active customer,Young adult (18-33),Low income,Parent,beverages
2,2,473747,1,3,3,12,21.0,196,1,1,...,2/17/2019,3,married,40423,South,active customer,Young adult (18-33),Low income,Parent,beverages
3,3,2254736,1,4,4,7,29.0,196,1,1,...,2/17/2019,3,married,40423,South,active customer,Young adult (18-33),Low income,Parent,beverages
4,4,431534,1,5,4,15,28.0,196,1,1,...,2/17/2019,3,married,40423,South,active customer,Young adult (18-33),Low income,Parent,beverages


In [49]:
# create crosstab between income_group and department
income_group_preferences = pd.crosstab(df_profiles['income_group'], df_profiles['department'], dropna = False)

In [50]:
income_group_preferences

department,alcohol,babies,bakery,beverages,breakfast,bulk,canned goods,dairy eggs,deli,dry goods pasta,...,household,international,meat seafood,missing,other,pantry,personal care,pets,produce,snacks
income_group,,,,,,,,,,,,,,,,,,,,,
High income,41099,105771,290807,619356,163094,7550,266276,1316016,251347,213214,...,177611,65446,183165,16365,8723,457860,106314,24975,2293263,602852
Low income,36650,82803,261763,737043,185892,9320,219841,1215904,245437,177781,...,181595,57089,125796,15742,8495,413041,106777,21593,2100660,953000
Medium income,66878,221818,568258,1215502,321864,16581,525957,2645262,507050,431141,...,340651,133456,365820,32661,17193,911804,211215,46492,4685350,1210554


In [51]:
# create crosstab between age_group and department
age_group_preferences = pd.crosstab(df_profiles['age_group'], df_profiles['department'], dropna = False)

In [52]:
age_group_preferences

department,alcohol,babies,bakery,beverages,breakfast,bulk,canned goods,dairy eggs,deli,dry goods pasta,...,household,international,meat seafood,missing,other,pantry,personal care,pets,produce,snacks
age_group,,,,,,,,,,,,,,,,,,,,,
Adult (34-49),35893,107219,281307,643821,168301,8530,253475,1307254,253138,204543,...,176455,64771,168602,16434,8417,447284,107054,22393,2291729,692921
Middle-Aged Adult(50-64),33841,95660,262218,607039,157306,7498,236038,1195871,234391,192942,...,163305,59173,156818,14781,8117,416682,100388,21543,2110443,648833
Senior Adult (65+),36693,106204,295997,684020,175296,8945,267509,1368072,265199,217316,...,186630,67199,181150,16915,8946,472383,111052,25115,2408104,728549
Young adult (18-33),38200,101309,281306,637021,169947,8478,255052,1305985,251106,207335,...,173467,64848,168211,16638,8931,446356,105812,24009,2268997,696103


In [53]:
#exporting df_profiles as customer profiles dataframe
df_profiles.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'customer_profiles.pkl'))